<a href="https://colab.research.google.com/github/turtletear/tubes-skc/blob/main/decision_tree_SKC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xlrd
import csv
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "drive/My Drive/SKC"

dt_datasets.csv  KDDCup99.csv  kddcup99_csv.csv  kddcup.data


In [ ]:
class config:
    kdd99train='drive/MyDrive/SKC/kddcup.data'
    kdd99test='drive/MyDrive/SKC/KDDCup99.csv'
def attack_class(train):
    train.loc[(train['label'].isin(['back.','land.','neptune.','pod.','smurf.','teardrop.'])),'label_type']='DOS'
    train.loc[train['label'].isin(['ipsweep.','nmap.','portsweep.','satan.']),'label_type']='Probing'
    train.loc[train['label'].isin(['ftp_write.','guess_passwd.','imap.','multihop.','phf.','spy.','warezclient.','warezmaster.']),'label_type']='R2L'
    train.loc[train['label'].isin(['buffer_overflow.','loadmodule.','perl.','rootkit.']),'label_type']='U2R'
    train.loc[train['label']=='normal.','label_type']='Normal'
    return train

def attack_class2(train):
    train.loc[(train['label'].isin(['back','land','neptune','pod','smurf','teardrop'])),'label_type']='DOS'
    train.loc[train['label'].isin(['ipsweep','nmap','portsweep','satan']),'label_type']='Probing'
    train.loc[train['label'].isin(['ftp_write','guess_passwd','imap','multihop','phf','spy','warezclient','warezmaster']),'label_type']='R2L'
    train.loc[train['label'].isin(['buffer_overflow','loadmodule','perl','rootkit']),'label_type']='U2R'
    train.loc[train['label']=='normal','label_type']='Normal'
    return train


cleanup_nums = {"protocol_type":     {"tcp": 1, "icmp": 2, "udp": 3},
                "service": {"vmnet": 1, "smtp": 2, "ntp_u":3, "shell":4, "kshell":5, "aol":6, "imap4":7, "urh_i":8, "netbios_ssn":9,
                           "tftp_u":10, "mtp":11, "uucp":12, "nnsp":13, "echo":14, "tim_i":15, "ssh":16, "iso_tsap":17, "time":18,
                           "netbios_ns":19,"systat":20, "hostnames":21, "login":22, "efs":23, "supdup":24, "http_8001":25, "courier":26,
                           "ctf":27,"finger":28,"nntp":29,"ftp_data":30,"red_i":31,"ldap":32,"http":33,"ftp":34,"pm_dump":35,"exec":36,
                           "klogin":37,"auth":38,"netbios_dgm":39,"other":40,"link":41,"X11":42,"discard":43,"private":44,"remote_job":45,
                           "IRC":46,"daytime":47,"pop_3":48,"pop_2":49,"gopher":50,"sunrpc":51,"name":52,"rje":53,"domain":54,"uucp_path":55,
                           "http_2784":56,"Z39_50":57,"domain_u":58,"csnet_ns":59,"whois":60,"eco_i":61,"bgp":62,"sql_net":63,"printer":64,
                           "telnet":65,"ecr_i":66,"urp_i":67,"netstat":68,"http_443":69,"harvest":70},
               "flag":{"RSTR":1,"S3":2,"SF":3,"RSTO":4,"SH":5,"OTH":6,"S2":7,"RSTOS0":8,"S1":9,"S0":10,"REJ":11},
               "label_type":{"Normal":1,"DOS":2,"Probing":3,"R2L":4,"U2R":5}}
attack_map={"label_type":{"Normal":1,"DOS":2,"Probing":3,"R2L":4,"U2R":5}}

column = ["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins","logged_in","lnum_compromised","lroot_shell","lsu_attempted","lnum_root","lnum_file_creations","lnum_shells","lnum_access_files","lnum_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [ ]:
#print("train data loading")
train = pd.read_csv(config.kdd99train)
train.columns= column
train.insert(0,'label_type','NAN')
# train.info()
# train.head()
# train['label'].value_counts()
# train['label_type'].value_counts()
#Mapping of Categorical data in the dataset


datapre = train
#Replacing the encoded data in the dataset
datapre.replace(cleanup_nums, inplace=True)

train = attack_class(datapre)

train['label_type'].value_counts()

# df[['two', 'three']] = df[['two', 'three']].astype(float)
train.replace(attack_map, inplace=True)
# train.head()
train.drop('label', axis = 1, inplace = True)
train.head()

,label_type,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,lroot_shell,lsu_attempted,lnum_root,lnum_file_creations,lnum_shells,lnum_access_files,lnum_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,1,0,1,33,3,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0
1,1,0,1,33,3,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0
2,1,0,1,33,3,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0
3,1,0,1,33,3,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0
4,1,0,1,33,3,238,1282,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0


In [ ]:
print("test data loading")
test = pd.read_csv(config.kdd99test)
test.insert(0,'label_type','NAN')
# test.info()
# test.head()
# test['label'].value_counts()

datapre = test
# datapre.transpose()
#Replacing the encoded data in the dataset
datapre.replace(cleanup_nums, inplace=True)
# datapre.transpose()
# datapre.head()
# datapre.head()
test = attack_class2(datapre)

test['label_type'].value_counts()

# df[['two', 'three']] = df[['two', 'three']].astype(float)
test.replace(attack_map, inplace=True)
# test.head()
test.drop('label', axis = 1, inplace = True)
test.head()

test data loading


,label_type,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,lroot_shell,lsu_attempted,lnum_root,lnum_file_creations,lnum_shells,lnum_access_files,lnum_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,1,0,1,33,3,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,1,0,1,33,3,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,1,0,1,33,3,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,1,0,1,33,3,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,1,0,1,33,3,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898430 entries, 0 to 4898429
Data columns (total 42 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   label_type                   int64  
 1   duration                     int64  
 2   protocol_type                int64  
 3   service                      int64  
 4   flag                         int64  
 5   src_bytes                    int64  
 6   dst_bytes                    int64  
 7   land                         int64  
 8   wrong_fragment               int64  
 9   urgent                       int64  
 10  hot                          int64  
 11  num_failed_logins            int64  
 12  logged_in                    int64  
 13  lnum_compromised             int64  
 14  lroot_shell                  int64  
 15  lsu_attempted                int64  
 16  lnum_root                    int64  
 17  lnum_file_creations          int64  
 18  lnum_shells                  int64  
 19  

In [ ]:
#Cross validation
# from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

train_data = train.values
features_train = train_data[:,1:42]
labels_train = train_data[:,0]

test_data = test.values
features_test = test_data[:,1:42]
labels_test = test_data[:,0]


# Decision Tree

In [ ]:
from sklearn import metrics
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import mean_squared_error


In [ ]:
#Decision Tree Classifier
clf = DecisionTreeClassifier(criterion="entropy", max_depth=7)
score = clf.fit(features_train, labels_train)
predict = clf.predict(features_test)

In [ ]:
#Confussion Matrix
matrix = confusion_matrix(labels_test, predict,)

print(matrix)

[[ 97197      3     28     49      0]
 [    26 391430      2      0      0]
 [    50      1   4056      0      0]
 [   626      0      0    498      2]
 [    28      0      2      0     22]]


In [ ]:
from sklearn.metrics import f1_score as f1
from sklearn.metrics import accuracy_score as acc_score
from sklearn.metrics import precision_score as prec
from sklearn.metrics import recall_score as recall

#evaluasi
print("Accuracy  : ", acc_score(labels_test,predict)*100, '%')
print("F1-Score  : ", f1(labels_test,predict, average='macro')*100, '%')
print("Precision : ", prec(labels_test,predict, average='macro')*100, '%')
print("Recall    : ", recall(labels_test,predict, average='macro')*100, '%')

Accuracy  :  99.83462208007772 %
F1-Score  :  83.19933157930943 %
Precision :  96.2358920408531 %
Recall    :  77.04077426464725 %


# Naive Bayes

In [ ]:
from sklearn import metrics
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error


In [ ]:
#Decision Tree Classifier
clf = GaussianNB()

score = clf.fit(features_train, labels_train)
predict = clf.predict(features_test)

In [ ]:
#Confussion Matrix
matrix = confusion_matrix(labels_test, predict)
print(matrix)

[[ 65254  27200    783     96   3944]
 [  2310 388900      2      0    246]
 [   975   2617     76      0    439]
 [   298     37     96      5    690]
 [     7      4      0      0     41]]


In [ ]:
from sklearn.metrics import f1_score as f1
from sklearn.metrics import accuracy_score as acc_score
from sklearn.metrics import precision_score as prec
from sklearn.metrics import recall_score as recall

#evaluasi
print("Accuracy  : ", acc_score(labels_test,predict)*100, '%')
print("F1-Score  : ", f1(labels_test,predict, average='macro')*100, '%')
print("Precision : ", prec(labels_test,predict, average='macro')*100, '%')
print("Recall    : ", recall(labels_test,predict, average='macro')*100, '%')

Accuracy  :  91.95498157969313 %
F1-Score  :  35.97856616932902 %
Precision :  40.2624167012361 %
Recall    :  49.51357061536904 %
